In [1]:
import networkx as nx
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import regex as re
import copy
#import community
import seaborn as sns
import pickle as pkl
import seaborn as sns
import pickle

DATA_PATH = 'data/'

# Preprocessing

# Preprocess the two book dataframes
- Load dataframes ('https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home', 'https://github.com/malcolmosh/goodbooks-10k-extended/blob/master/README.md')
- Remove faulty elements of dataframe without the descriptions
- Remove "additions" to titles in the description dataframes - Example: title (series #1) -> title
- Make a new dataframe that contains "Book ID" (from the dataframe without descriptions), "Book Title" (-||-), "genre" (description dataframe), "description" (description dataframe)
- Store this dataframe

In [ ]:
# Load dataframes
descriptions_df = pd.read_csv(DATA_PATH + "books_descriptions.csv")
book_ID_df = pd.read_json(DATA_PATH + 'goodreads_book_works.json', lines=True)

# Remove columns that are not needed in book_ID_df
book_ID_df = book_ID_df.filter(items=["best_book_id", "original_title", "reviews_count"])


In [ ]:
# List amount of books are in each dataframe
print(f"Amount of books in descriptions_df: {len(descriptions_df)}")
print(f"Amount of books in book_ID_df: {len(book_ID_df)}")

In [ ]:
# Remove faulty elements from the book_ID_df
book_ID_df = book_ID_df[book_ID_df['original_title'] != '']

print(f"Amount of books in book_ID_df: {len(book_ID_df)}")

In [ ]:
# Remove faulty elements from the description dataframe
descriptions_df = descriptions_df[descriptions_df['description'].apply(lambda x: isinstance(x, str))]

In [ ]:
# Remove "additions" to titles in the descriptions_df
for i, row in tqdm(descriptions_df.iterrows()):
    original_title = row["title"]
    new_title = re.sub(r'\((.*)', '', original_title)
    descriptions_df.at[i, "title"] = new_title.strip()

# Check if it worked -> it did
# descriptions_df.head()

In [ ]:
# Lower case all titles to not have confusion in this manner
descriptions_df['title'] = descriptions_df['title'].str.lower()
book_ID_df['original_title'] = book_ID_df['original_title'].str.lower()

In [ ]:
# Find corresponding indexes to merge the dataframes
warnings.simplefilter(action='ignore', category=FutureWarning)
titles_not_found = []
book_df = pd.DataFrame(columns=['book_id', 'title', 'description', 'genres'])

for i, row in tqdm(descriptions_df.iterrows(), total = descriptions_df.shape[0]):
    title = row['title']
    # Check if the title is in book_ID_df, else append it to the titles_not_found list
    if title in book_ID_df['original_title'].values:
        # Get all rows that have the a matching title as the current row
        temp_df = book_ID_df[book_ID_df['original_title'] == title]

        # Get the book id of the book  with the highest amount of reviews
        book_id = temp_df['best_book_id'][temp_df['reviews_count'].idxmax()]
        descriptions = row['description']
        genres = row['genres']
        book_df = book_df.append({'book_id': book_id, 'title': title, 'description': descriptions, 'genres': genres}, ignore_index=True)
    else:
        titles_not_found.append(title)

# print the amount of elements that are not found
print(f"Out of the 10000 titles {len(titles_not_found)} are not found in the book_ID_df")

In [ ]:
# Somehow the same book appears multiple times, hence we drop the duplicates
book_df.drop_duplicates(subset=['title'], inplace=True)

In [1]:
# TODO - Possibly add the capacity to look through titles and determine if a shorter version could be found in the other dataset ("harry potter and the sorcersers stone" becoming "harry potter and the " and possibly finding harry potter and the philosophers stone"

# Preprocess the shelves
- Use "book_id_map.csv" to find the books we use (ids) and store "new_ids" (the ids we can use to find the relevant shelfs)
- Drop all rows in "goodreads_interactions.csv" that have different ids than "new_ids".
- Store this dataset.

In [2]:
# Load dataframes
book_id_map_df = pd.read_csv(DATA_PATH + "book_id_map.csv")
book_df = pd.read_csv(DATA_PATH + "book_df.csv")

In [ ]:
# Create map from book_id to book_id_csv
book_id_map = {book_id_map_df['book_id'][i]: book_id_map_df['book_id_csv'][i] for i in range(len(book_id_map_df))}

# Change the book ID in our dataset to match the shelf dataset
remove_list = [] # remove about 15 books that are for inexplicable reasons not in the shelf dataset
for i in range(len(book_df)):
    try:
        book_df["book_id"][i] = book_id_map[book_df["book_id"][i]]
    except:
        remove_list.append(i)
print(f"Out of {i} books {len(remove_list)} are not in the shelf dataset and hence removed")
book_df.drop(remove_list, inplace=True)

# Save the book_df dataframe with the index change
book_df.to_csv(DATA_PATH + 'book_matching_ids_df.csv', index=False)

In [ ]:
# Load the shelves dataframe (this takes some time and memory)
shelves_df = pd.read_csv(DATA_PATH + "goodreads_interactions.csv")

# Check how many books are on the shelves of ALL users combined
print(f'There are {len(shelves_df)} books, copies counted aswell')

In [ ]:
# Remove books on the shelves that are not in the book_df
shelves_df = shelves_df[shelves_df['book_id'].isin(book_df['book_id'].tolist())]

# Save the new shelves_df
shelves_df.to_csv(DATA_PATH + 'shelves_df.csv', index=False)

# Check how many books are on the shelves of ALL users combined - after removal of books not in book_df
print(f"We have {len(shelves_df)} shelves in total, and in these there are {len(set(shelves_df['book_id'].tolist()))} unique books.")

# TF-IDF embeddings
- Create TF-IDF embeddings
- Create them for genres aswell

In [ ]:
# imports for the text analysis
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import ast

# Import the book description dataframe
book_df = pd.read_csv(DATA_PATH + "book_matching_ids_df.csv") # TODO help me not make a "Unnamed: 0" column... I want to use the book_id as index, but then it creates this column

# Logorithmic scale chosen for IDF
BASE = 2

In [2]:
# Function to clean strings (from week 7)
def clean_strings(strings):
    """ Cleans a list of strings by removing URLs, numbers, punctuation and stop words

    Args:
    - strings: a list of strings

    returns:
    - cleaned_strings: a list of cleaned strings
    """
    cleaned_strings = []
    stop_words = set(stopwords.words('english'))

    for string in strings:
        # Remove URLs
        string = re.sub(r'http\S+', '', string)

        # Remove numbers
        string = re.sub(r'[0-9]', '', string)

        # Keep only what is not punctuation
        string = re.sub(r'[^\w\s]', '', string)

        # Lowercase
        string = string.lower()

        # Remove empty strings and remove stop words and moke them
        if len(string) and string not in stop_words and type(string) == str:
            cleaned_strings.append(string)

    return cleaned_strings


In [ ]:
# Create tokens from descriptions
# If the dataframe has not yet gotten the tokens, do it here and save it
if not 'tokens' in book_df.columns:
    book_df['tokens'] = None
    for i, row in tqdm(book_df.iterrows(), total = book_df.shape[0]):
        description = row['description']
        # If the description is not a string, it is probably a NaN, so we set it to None
        if type(description) == str:
            tokens = nltk.word_tokenize(description)
            clean_tokens = clean_strings(tokens)
            book_df['tokens'][i] = clean_tokens # str(clean_tokens)

    book_df.to_csv(DATA_PATH + 'book_matching_ids_df.csv', index=False)
else:
    book_df['tokens'] = book_df['tokens'].apply(ast.literal_eval)

In [ ]:
# Calculate the tf scores for each community (from the previous weeks)
def TF_from_corpus(corpus):
    """ Calculates the TF scores for each word in the corpus

    Args:
        corpus (list): list of lists of words/strings

    Returns:
        TF_df (pandas.DataFrame): Dataframe containing the TF scores for each word in the corpus
    """
    # Create empty dictionary to keep track of word counts
    word_counts = {}
    n_communities = len(corpus)

    # Iterate through all communities
    for i, document in tqdm(enumerate(corpus), total=n_communities):
        # Iterate through each word in the current sublist
        for word in document:
            # If the current word is not in the dictionary, add it with a list of zeros
            if word not in word_counts:
                word_counts[word] = [0] * n_communities

            # Increment count for the current word and list index
            word_counts[word][i] += 1

    # Create pandas dataframe from the word_counts dictionary
    TF_df = pd.DataFrame.from_dict(word_counts).transpose()

    return TF_df

In [ ]:
# Function that takes in a TF and an IDF and computes the TF_IDF dataframe (from the previous weeks)
def make_TF_IDF(TF_df, IDF_dict):
    """Multiply the TF and IDF scores to get the TF-IDF scores

    Args:
        TF_df (pandas.DataFrame): Dataframe containing the TF scores for each word in the corpus
        IDF_dict (dict): Dictionary containing the IDF scores for each word in the corpus

    Returns:
        TF_IDF (pandas.DataFrame): Dataframe containing the TF-IDF scores for each word in the corpus
    """
    # Create the TF-IDF dataframe
    TF_IDF = pd.DataFrame(index=TF_df.index, columns=TF_df.columns)

    # iterate over the index of the DataFrame
    for word in tqdm(TF_df.index, total=TF_df.shape[0]):
        # multiply the values by the IDF_dict value
        TF_IDF.loc[word] = TF_df.loc[word] * IDF_dict[word]


    return TF_IDF

In [ ]:
# Load the TF dataframe for the corpus if possible, else create it
try:
    TF_book_df = pd.read_csv(DATA_PATH + "TF_book_df.csv", index_col=0)
except:
    # Create it
    TF_book_df = TF_from_corpus(book_df['tokens'])

    # Rename the columns to the book_ids
    TF_book_df.columns = book_df['book_id'].tolist()

    # Save the TF dataframe
    TF_book_df.to_csv(DATA_PATH + "TF_book_df.csv")

In [ ]:
# Create the total token count "T_all_books" and the IDF score for each book "IDF_book_dict"
try:
    IDF_dict = np.load(DATA_PATH + 'IDF_dict.npy', allow_pickle=True).item()
except:
    T_all_books = TF_book_df.apply(lambda row: (row != 0).sum(), axis=1)
    # The log BASE is chosen when loading the libraries
    IDF_dict = {word: np.emath.logn(BASE, len(TF_book_df.columns)/ T_all_books[word]) for word in TF_book_df.index}

    np.save(DATA_PATH + 'IDF_dict.npy', IDF_dict)

In [ ]:
# Create the TF-IDF scores for each book "TF_IDF_book_df" if it has not already been made
try:
    TF_IDF_book_df = pd.read_csv(DATA_PATH + "TF_IDF_book_df.csv", index_col=0)
except:
    # Create the dataframe
    TF_IDF_book_df = make_TF_IDF(TF_book_df, IDF_dict)

    # Save the dataframe
    TF_IDF_book_df.to_csv(DATA_PATH + "TF_IDF_book_df.csv")

#### Genre TF and TF_IDF scores
- Do this by having all books with a genre define the "document" for that genre
- Then compute the "TF_genre_df" dataframe, by summing all books from "TF_book_df" from that genre
- Here we make the decision that the IDF is the same as for the books.
    - (Alternatively one could have weighed each book and made a new IDF score, however, this weighs a book with twice as many genres twice as large, hence we use the other option)

In [ ]:
# Find the set of genres
genres = set()
for i in book_df["genres"].to_list():
    genres = genres.union(set(ast.literal_eval(i)))

In [ ]:
# Try to load the TF_genres_df, else create it
try:
    # Load the TF_genres_df
    TF_genres_df = pd.read_csv(DATA_PATH + "TF_genres_df.csv", index_col=0)
except:
    # For each genre, sum all TF scores for books in that genre
    TF_genres_df = pd.DataFrame(index=TF_book_df.index, columns=genres)

    TF_genres_df = TF_genres_df.fillna(0)

    # Go through all books and add the TF scores to the genres of the book
    for i, row in tqdm(book_df.iterrows()):
        for genre in ast.literal_eval(row['genres']):
            TF_genres_df[genre] = TF_genres_df[genre] + TF_book_df[row['book_id']]

    # Save the genres_TF_df
    TF_genres_df.to_csv(DATA_PATH + "TF_genres_df.csv")

In [ ]:
# Try to load the TF_IDF_genres_df, else create it
try:
    # Load the TF_IDF_genres_df
    TF_IDF_genres_df = pd.read_csv(DATA_PATH + "TF_IDF_genres_df.csv", index_col=0)
except:
    # Create the dataframe
    TF_IDF_genres_df = make_TF_IDF(TF_genres_df, IDF_dict)

    # Save the dataframe
    TF_IDF_genres_df.to_csv(DATA_PATH + "TF_IDF_genres_df.csv")

In [3]:
# Have not added things with # delete

# Create a genre for each book
- make the inner product which each book and the genre vector (both normed)
- let the largest inner product that the book contains be the genre of the book

In [4]:
# Inner product function
def inner_product(v1, v2):
    """Calculates the normed inner product of two vectors

    Args:
        v1 (list): list of numbers
        v2 (list): list of numbers

    Returns:
        inner_product (float): inner product of the two vectors (divided by the product of their norms)
    """

    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [5]:
# Generate genre by taking inner products between each book and each genre of that book, and choosing the maximal
def get_genres(book_df, TF_IDF_genres_df, TF_IDF_book_df):
    # TODO: Check these args and return
    """
    Args:
        book_df (DataFrame): DataFrame with genres for each book
        TF_IDF_genres_df (DataFrame): DataFrame with TF_IDF each genre
        TF_IDF_book_df (DataFrame): DataFrame with TF_IDF each book

    Returns:
        genres (dict): Dictionary with book ids as keys and the corresponding gerne that matches the best
    """

    # Create a dictionary to store the genre for each book
    genres = {}

    # Go through each book
    for i, row in tqdm(book_df.iterrows()):
        # Get the TF-IDF scores for the current book
        book_TF_IDF = TF_IDF_book_df[row["book_id"]]

        best_genre = (None, 0)

        # Go through each genre of the book
        for genre in ast.literal_eval(row['genres']):
            genre_TF_IDF = TF_IDF_genres_df[genre]
            # print(f"inner_product {inner_product(book_TF_IDF, genre_TF_IDF)} {genre}") # Testing
            if inner_product(book_TF_IDF, genre_TF_IDF) > best_genre[1]:
                best_genre = (genre, inner_product(book_TF_IDF, genre_TF_IDF))
        # print(best_genre) # Testing
        # break # Testing
        # Save the best genre
        genres[row["book_id"]] = best_genre[0]

    return genres

In [ ]:
# Get the genres for each book
genres = get_genres(book_df, TF_IDF_genres_df, TF_IDF_book_df)

book_df["top_genre"] = book_df["book_id"].map(genres)


In [ ]:
# Save the book_df as complete_book_df
book_df.to_csv(DATA_PATH + "complete_book_df.csv")

# Create network
- Load dataframe with shelves
- Implode dataframe to make shelves into edgelist
- Calculate assortativity
    - Genre
    - Degrees
- Get largest sub network
- Create communities
    - Color according to genre

In [1]:
# Create the network
def create_network(threshhold=0.2):
    """
    Creates the network from the shelf edges.
    If the edges have already been created, they are loaded.
    Otherwise, they are created and stored. The edges are filtered by the threshhold. The network is returned.

    Args:
        threshhold (int): The minimum number of times an edge must appear to be included in the network.

    Returns:
        G (networkx.Graph): The network of book edges.
    """
    # If the edges have already been created, load them
    try:
        # Load the shelf edges
        edges = np.load(DATA_PATH + 'shelf_edges.npy', allow_pickle=True).item() # (takes about 10 minutes)
    except:
        # Create the shelf edges and store them (this takes about 12 hours)
        shelves_df = pd.read_csv(DATA_PATH + 'shelves_df.csv')
        # Collapse into shelves
        imploded_df = shelves_df.groupby('user_id')['book_id'].apply(list).reset_index()

        # Count occurrences of each book
        book_counts = {}
        for i, shelf in tqdm(imploded_df.iterrows(), total=imploded_df.shape[0]):
            for book in shelf['book_id']:
                if book in book_counts:
                    book_counts[book] += 1
                else:
                    book_counts[book] = 1

        # Count appearances of pairs of books
        edges = {}
        for i, shelf in tqdm(imploded_df.iterrows(), total=imploded_df.shape[0]):
            if len(shelf['book_id']) > 1:
                # Generate pairs of books
                for i in range(len(shelf['book_id'])):
                    for j in range(i+1, len(shelf['book_id'])):
                        # Create edge
                        edge = frozenset([shelf['book_id'][i], shelf['book_id'][j]])
                        # Add edge to dictionary
                        if edge in edges:
                            edges[edge] += 1
                        else:
                            edges[edge] = 1

        # Weigh edges by the inverse of the root of the number of times each book appears
        for edge in edges:
            edges[edge] = edges[edge] / (book_counts[list(edge)[0]] * book_counts[list(edge)[1]])**0.5

        # Store the edges
        np.save(DATA_PATH + 'shelf_edges.npy', edges)

    # Remove edges that are below the threshhold
    edges = {k: v for k, v in edges.items() if v >= threshhold}

    # Create the network
    G = nx.Graph()
    G.add_edges_from(edges.keys())

    return G


In [ ]:
#TODO: omskrevet til at læse/skrive netværk i steder for pickle
# Load/Generate the network
try:
    # Load the network
    G = nx.read_graphml(DATA_PATH + "shelves_network_04.graphml")
except:
    # Make the network
    G = create_network() #TODO: stod før 2000 her i ()

    # Save the network
    nx.write_graphml(G, DATA_PATH + "shelves_network.graphml")


In [ ]:
# load data
if shelves_df not in locals():
    shelves_df = pd.read_csv(DATA_PATH + 'shelves_df.csv')

In [ ]:
# Implode dataframe to get books for each user in a list for each
imploded_df = shelves_df.groupby('user_id')['book_id'].apply(list).reset_index()
print(f'The amount of users are: {len(imploded_df)}')

In [ ]:
# Create dictionary edges and appearances
edges = {}

# Go through shelves to get edges
for i, shelf in tqdm(imploded_df.iterrows(), total=imploded_df.shape[0]):
    if len(shelf['book_id']) > 1:
        # Generate pairs of books
        for i in range(len(shelf['book_id'])):
            for j in range(i+1, len(shelf['book_id'])):
                # Create edge
                edge = frozenset([shelf['book_id'][i], shelf['book_id'][j]])
                # Add edge to dictionary
                if edge in edges:
                    edges[edge] += 1
                else:
                    edges[edge] = 1

print(f'The amount of edges are {len(edges)}')

In [ ]:
# TODO: Comment on this being to much in rapport or that is was to much (before threshold)

In [ ]:
# Save the edges
np.save(DATA_PATH + 'shelf_edges.npy', edges)

In [ ]:
#TODO: Jeg tror ikke at det her er ordenligt pushet eftersom det er en pickle og ikke en npy, og har derfor heller ikke tilføjet resten af "Generate network"

In [ ]:
# Load the shelf edges
edges = np.load(DATA_PATH + 'shelf_edges.npy', allow_pickle=True).item()

In [ ]:
# Plot a histogram of the number of times each edge appears
plt.hist(edges.values(), log=True, bins=100)
# plt.xscale('log')
plt.xlabel('People who had booth books')
plt.ylabel('Count')
plt.title('Distribution of books appearing together')
plt.show()

In [2]:
#TODO: noget threshold jeg ikke forstår da det er med i funktionen "create_network" skal bare have hjælp herfra

In [ ]:
# Count occurrences of each book
book_counts = {}
for i, shelf in tqdm(imploded_df.iterrows(), total=imploded_df.shape[0]):
    for book in shelf['book_id']:
        if book in book_counts:
            book_counts[book] += 1
        else:
            book_counts[book] = 1

In [ ]:
# Plot a histogram of the number of times each book appears
plt.hist(book_counts.values(), log=True, bins=100)
# plt.xscale('log')
plt.xlabel('People who had the book')
plt.ylabel('Count')
plt.title('Distribution of books appearing together')

In [ ]:
#TODO: Show how to make communities

# Visualize graph

In [2]:
import netwulf
import ast

In [3]:
# Used to make all node ids ints
def make_all_nodes_ints(graph):
    #TODO: args and returns

    # Check if first id is a string, in which case, don't do anyting
    if isinstance(list(graph.nodes())[0], int):
            return graph

    # Make a mapping so each node can be renamed into what it was, but a string instead
    node_map = {}
    for node in graph.nodes():
        node_map[node] = int(node)
    str_graph = nx.relabel_nodes(graph, node_map)

    return str_graph

In [12]:
# Load graphs
shelf_graph = nx.read_graphml(DATA_PATH + 'shelves_graph_04.graphml')
shelf_graph = make_all_nodes_ints(shelf_graph)

NLP_graph = nx.read_graphml(DATA_PATH + 'NLP_graph_04.graphml')
NLP_graph = make_all_nodes_ints(NLP_graph)

#Load communities
shelf_louvain = np.load(DATA_PATH + 'shelves_communities_04.npy', allow_pickle = True)
NLP_louvain = np.load(DATA_PATH + 'NLP_communities_04.npy', allow_pickle= True)

In [5]:
if not "complete_book_df" in locals():
    complete_book_df = pd.read_csv(DATA_PATH + "complete_book_df.csv")

In [13]:
def make_attributes(df, graph):
    #TODO: args and returns
    # make a dictionary with attributes for each node
    book_attributes = dict()
    for i, book in tqdm(df.iterrows(), total=df.shape[0]):
        node = str(book['book_id'])
        if not node in graph.nodes():
            continue
        book_attributes[node] = dict()
        top_genre = book['top_genre']
        title = book['title']
        genres = ast.literal_eval(book['genres'])

        book_attributes[node]['title'] = title
        book_attributes[node]['genres'] = genres
        book_attributes[node]['top_genre'] = top_genre
    return book_attributes

In [14]:
shelf_attribute_dict = make_attributes(complete_book_df, shelf_graph)
NLP_attribute_dict = make_attributes(complete_book_df, NLP_graph)

100%|██████████| 7676/7676 [00:00<00:00, 21288.14it/s]


In [15]:
nx.set_node_attributes(shelf_graph, shelf_attribute_dict)
nx.set_node_attributes(NLP_graph, NLP_attribute_dict)

In [16]:
import matplotlib.colors as mcolors
# List of colors to use for the visualization


# Use netwulf to visualize the communities
def visualize_communities(graph, communities):
    colors = list(mcolors.CSS4_COLORS.values())[20:len(communities) + 20]
    # Create a dictionary that maps nodes to communities
    node_to_community = {}
    for i, community in enumerate(communities):
        for node in community:
            node = str(node)
            node_to_community[node] = i

    # Add a color attribute to the nodes
    for node in graph.nodes:
        # Add the community number as a node attribute
        graph.nodes[node]["color"] = colors[node_to_community[node]]

    # Visualize the graph with netwulf
    netwulf.visualize(graph, config={"Node color": "color",
                                    'node_size': 50,
                                    'link_width': 0.1,
                                    'link_alpha': 0.01,
                                    'node_size_variation': 0.3,
                                    'zoom': 0.65,
                                    'scale_node_size_by_strength': True,})

In [64]:
visualize_communities(shelf_graph, shelf_louvain)

In [19]:
visualize_communities(NLP_graph, NLP_louvain)

### Colors for genre

In [24]:
def create_genre_dict(graph):
    genre_dict = {}
    for node in tqdm(graph.nodes):
        genre = nx.get_node_attributes(graph, 'top_genre')[node]
        try:
            genre_dict[genre].append(node)
        except:
            genre_dict[genre] = [node]
    return genre_dict

In [25]:
shelf_genre_dict = create_genre_dict(shelf_graph)
NLP_genre_dict = create_genre_dict(NLP_graph)

100%|██████████| 7671/7671 [00:12<00:00, 611.70it/s]


In [26]:
# Use netwulf to visualize the genres
def visualize_genres(graph, genre_dict):
    colors = list(mcolors.CSS4_COLORS.values())[20:len(genre_dict.keys())+20]
    # Create a dictionary that maps nodes to genre
    node_to_genre = {}
    for i, genre in enumerate(genre_dict.keys()):
        for node in genre_dict[genre]:
            node_to_genre[node] = i



    # Add a color attribute to the nodes
    for node in graph.nodes:
        # Add the community number as a node attribute
        graph.nodes[node]["color"] = colors[node_to_genre[node]]

    # Visualize the graph with netwulf
    netwulf.visualize(graph, config={"Node color": "color",
                                    'node_size': 50,
                                    'link_width': 0.1,
                                    'link_alpha': 0.01,
                                    'node_size_variation': 0.3,
                                    'zoom': 0.65,
                                    'scale_node_size_by_strength': True,})

In [62]:
visualize_genres(shelf_graph, shelf_genre_dict)

In [27]:
visualize_genres(NLP_graph, NLP_genre_dict)

# Generate wordclouds

In [30]:
# Find top three books for each community according to degree
def get_top_3_books(graph_type, louvain_groups):
    top_3_books = {}
    # Store the top 3 books by degree for all communities
    for i, community in enumerate(louvain_groups):

        # Get the top 3 books by degree
        sorted_dict = sorted(dict(graph_type.degree(community)).items(), key=lambda x: x[1], reverse=True)
        names = nx.get_node_attributes(graph_type, "title")
        top_3_keys = [k for k, v in sorted_dict[:3]] # Get the ID
        top_3_names = [names[k] for k in top_3_keys] # get the name

        top_3_books[i] = [(top_3_keys[j], top_3_names[j]) for j in range(len(top_3_keys))]
        return top_3_books

Shelf graph
**********
1. largest community 1: 
Book: the hunger games, degree: 7312
Book: to kill a mockingbird, degree: 6527
Book: catching fire, degree: 5683

2. largest community 0: 
Book: harry potter and the chamber of secrets, degree: 5694
Book: harry potter and the prisoner of azkaban, degree: 5530
Book: harry potter and the deathly hallows, degree: 5394

3. largest community 5: 
Book: eclipse, degree: 2309
Book: breaking dawn, degree: 2232
Book: city of bones, degree: 1947

4. largest community 3: 
Book: angels & demons, degree: 1024
Book: a time to kill, degree: 151
Book: along came a spider, degree: 145

5. largest community 2: 
Book: the riddle, degree: 1384
Book: a wrinkle in time, degree: 771
Book: charlotte's web, degree: 657

6. largest community 9: 
Book: fifty shades of grey, degree: 890
Book: beautiful disaster, degree: 496
Book: fifty shades darker, degree: 399

7. largest community 6: 
Book: dark lover, degree: 498
Book: dead until dark, degree: 405
Book: halfway t

In [36]:
shelf_int_graph = make_all_nodes_ints(shelf_graph)

In [25]:
shelf_top_3_books = get_top_3_books(shelf_int_graph, shelf_louvain)
#print(shelf_top_3_books)

In [26]:
shelf_top_3_books

{0: [(968, 'harry potter and the chamber of secrets'),
  (941, 'harry potter and the prisoner of azkaban'),
  (613, 'harry potter and the deathly hallows')]}